In [ ]:
# %% [code]
import xml.etree.ElementTree as ET
import numpy as np
import networkx as nx
import plotly.graph_objects as go
import math

def compute_segment_points(geom):
    """
    根据 geometry 节点判断类型（直线、arc、spiral）并返回采样点列表
    其中：
      - 对于直线 geometry，直接使用直线公式；
      - 对于 arc，利用 curvature 属性；
      - 对于 spiral，则采用简单的数值积分方法（本示例采用欧拉法）。
    """
    x = float(geom.get('x', '0'))
    y = float(geom.get('y', '0'))
    hdg = float(geom.get('hdg', '0'))
    length = float(geom.get('length', '0'))
    num_points = max(int(length // 0.5), 2)
    t_vals = np.linspace(0, length, num=num_points)
    
    arc_tag = geom.find('arc')
    spiral_tag = geom.find('spiral')
    
    if arc_tag is not None:
        # 圆弧：使用 curvature 属性计算采样点
        k = float(arc_tag.get('curvature'))
        points = []
        for s in t_vals:
            if abs(k) > 1e-8:
                dx = (np.sin(hdg + k * s) - np.sin(hdg)) / k
                dy = -(np.cos(hdg + k * s) - np.cos(hdg)) / k
            else:
                dx = s * np.cos(hdg)
                dy = s * np.sin(hdg)
            points.append((x + dx, y + dy))
        return points

    elif spiral_tag is not None:
        # 螺旋线：曲率从 curvStart 变化到 curvEnd，采用简单欧拉法积分
        curvStart = float(spiral_tag.get('curvStart'))
        curvEnd = float(spiral_tag.get('curvEnd'))
        points = []
        current_angle = hdg
        s_delta = length / (num_points - 1)
        current_x, current_y = x, y
        points.append((current_x, current_y))
        for i in range(1, num_points):
            s = i * s_delta
            # 当前曲率按线性插值
            k = curvStart + (curvEnd - curvStart) * (s / length)
            current_angle += k * s_delta  # 简单欧拉积分
            current_x += s_delta * np.cos(current_angle)
            current_y += s_delta * np.sin(current_angle)
            points.append((current_x, current_y))
        return points

    else:
        # 默认处理直线 geometry
        return [(x + t * np.cos(hdg), y + t * np.sin(hdg)) for t in t_vals]

def parse_oxdr_all(file_path):
    """
    解析 oxdr 文件，提取每个 road 的采样点、拓扑关系和 junction 属性。
    遍历所有 road 节点（XPath ".//road"）：
      - 从 planView 下读取 geometry 得到采样点；
      - 从 link 中提取 predecessor 与 successor 信息；
      - 从 road 元素读取 junction 属性（"-1" 表示正常路段，其它值表示所属 junction）。
    返回的 roads_data 为一个字典，键为 road_id，值为包含：
         "lane_points": [(x, y), ...],
         "predecessor": (elementType, elementId) 或 None,
         "successor": (elementType, elementId, contactPoint) 或 None,
         "junction": junction 字符串,
         "in_range": False
    同时针对 junction 路段，进行重复连接的去重处理：若多条 road 同属于同一 junction，
    且其前驱和后继（elementType 为 "road" 的情况）构成的无序集合相同，则只保留其中一条。
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    roads_data = {}
    for road in root.findall('.//road'):
        road_id = road.get('id')
        
        # 解析采样点
        lane_points = []
        plan_view = road.find('planView')
        if plan_view is not None:
            for geom in plan_view.findall('geometry'):
                seg_pts = compute_segment_points(geom)
                if lane_points:
                    lane_points.extend(seg_pts[1:])  # 避免重复
                else:
                    lane_points.extend(seg_pts)
                    
        # 提取 link 信息
        link = road.find('link')
        predecessor = None
        successor = None
        if link is not None:
            pred = link.find('predecessor')
            succ = link.find('successor')
            if pred is not None:
                predecessor = (pred.get('elementType'), pred.get('elementId'))
            if succ is not None:
                successor = (succ.get('elementType'), succ.get('elementId'), succ.get('contactPoint'))
                
        # 获取 junction 属性，默认"-1"
        junction = road.get('junction', "-1")
        
        roads_data[road_id] = {
            "lane_points": lane_points,
            "predecessor": predecessor,
            "successor": successor,
            "junction": junction,
            "in_range": False,
        }
    
    # 针对 junction 路段去重：对 junction != "-1" 的路段，
    # 根据 (junction, frozenset({predecessor_road_id, successor_road_id})) 分组，
    # 同一组中只保留第一个
    groups = {}
    for road_id, data in list(roads_data.items()):
        if data["junction"] != "-1":
            pre = data.get("predecessor")
            succ = data.get("successor")
            pre_id = pre[1] if pre is not None and pre[0]=="road" else None
            succ_id = succ[1] if succ is not None and succ[0]=="road" else None
            if pre_id is not None and succ_id is not None:
                conn_key = (data["junction"], frozenset({pre_id, succ_id}))
            else:
                conn_key = (data["junction"], None)
            groups.setdefault(conn_key, []).append(road_id)
    for key, road_list in groups.items():
        if len(road_list) > 1:
            # 保留第一个，其余删除
            for rid in road_list[1:]:
                roads_data.pop(rid)
    return roads_data
    
    # 针对 junction 路段进行去重：
    # 对于 junction != "-1" 的路段，按 (junction, {predecessor_road_id, successor_road_id}) 分组
    groups = {}
    for road_id, data in list(roads_data.items()):
        if data["junction"] != "-1":
            # 仅考虑 elementType 为 "road" 的连接（否则不参与去重）
            pre = data.get("predecessor")
            succ = data.get("successor")
            pre_id = pre[1] if pre is not None and pre[0]=="road" else None
            succ_id = succ[1] if succ is not None and succ[0]=="road" else None
            if pre_id is not None and succ_id is not None:
                # 无论前驱后继的顺序如何，均看作相同连接（使用 frozenset 实现无序）
                conn_key = (data["junction"], frozenset({pre_id, succ_id}))
            else:
                conn_key = (data["junction"], None)
            groups.setdefault(conn_key, []).append(road_id)
    
    # 对每组中多个 road，只保留一个（保留第一个出现的，其余删除）
    for key, road_list in groups.items():
        if len(road_list) > 1:
            for rid in road_list[1:]:
                roads_data.pop(rid)
    
    return roads_data


def extract_lanes_in_range(roads_data, current_pos, sensing_range):
    """
    遍历所有路段的采样点，判断是否在感知范围内，并更新 roads_data 中的 in_range 标志。
    简单策略：如果某个路段任一采样点与 current_pos 的欧氏距离小于 sensing_range，则认为该路段在感知范围内。
    """
    cx, cy = current_pos
    for road_id, road in roads_data.items():
        for (x, y) in road.get("lane_points", []):
            if np.linalg.norm([x - cx, y - cy]) <= sensing_range:
                road["in_range"] = True
                break

def generate_circle_points(center, radius, num_points=50):
    """
    生成圆的采样点，用于绘制感知区域。
    """
    cx, cy = center
    theta = np.linspace(0, 2 * np.pi, num_points)
    x_circle = (cx + radius * np.cos(theta)).tolist()
    y_circle = (cy + radius * np.sin(theta)).tolist()
    return x_circle, y_circle

def visualize_lanes(roads_data, current_pos, sensing_range):
    """
    使用 Plotly 可视化所有车道，
      - 根据 roads_data 每个路段的 lane_points 绘制路径，
      - 如果 in_range 为 True，则以蓝色绘制；否则以灰色绘制；
      - 绘制当前位置与感知区域。
    """
    fig = go.Figure()
    
    # 绘制所有路段
    for road_id, data in roads_data.items():
        lane = data.get("lane_points", [])
        if not lane:
            continue
        xs, ys = zip(*lane)
        color = 'blue' if data.get("in_range", False) else 'grey'
        fig.add_trace(go.Scatter(
            x=xs, y=ys,
            mode='lines',
            line=dict(color=color, width=2),
            name=f'路段 {road_id}'
        ))
    
    # 绘制当前位置：红色十字标记
    cx, cy = current_pos
    fig.add_trace(go.Scatter(
        x=[cx], y=[cy],
        mode='markers',
        marker=dict(color='red', size=12, symbol='x'),
        name='当前位置'
    ))
    
    # 绘制感知范围圆形区域（用红色虚线）
    circle_x, circle_y = generate_circle_points(current_pos, sensing_range)
    fig.add_trace(go.Scatter(
        x=circle_x, y=circle_y,
        mode='lines',
        line=dict(color='red', dash='dash'),
        name='感知范围'
    ))
    
    fig.update_layout(
        title='路径形状可视化',
        xaxis_title='X 坐标',
        yaxis_title='Y 坐标',
        legend_title='图例',
        xaxis=dict(scaleanchor="y", scaleratio=1),
        template="plotly_white"
    )
    fig.show()

def build_topology_graph(roads_data):
    """
    根据 roads_data 构建用于路径规划的 networkx 图。
    节点：每个 road 节点（road_id），节点属性包含 'pos'（取其采样点中间点）、'junction' 等信息；
    边：基于 link 中 elementType 为 "road" 的连接添加边，权重设置为两节点中间点之间的欧氏距离。
    """
    G = nx.DiGraph()
    for road_id, data in roads_data.items():
        pts = data.get("lane_points", [])
        if pts:
            mid = pts[len(pts)//2]
        else:
            mid = (0, 0)
        G.add_node(road_id, pos=mid, junction=data.get("junction"), road_id=road_id)
    for road_id, data in roads_data.items():
        pred = data.get("predecessor")
        succ = data.get("successor")
        if pred is not None:
            etype, eid = pred
            if etype == "road" and eid in roads_data:
                pos1 = G.nodes[eid]['pos']
                pos2 = G.nodes[road_id]['pos']
                weight = np.linalg.norm(np.array(pos1) - np.array(pos2))
                G.add_edge(eid, road_id, weight=weight)
        if succ is not None:
            etype, eid, cp = succ
            if etype == "road" and eid in roads_data:
                pos1 = G.nodes[road_id]['pos']
                pos2 = G.nodes[eid]['pos']
                weight = np.linalg.norm(np.array(pos1) - np.array(pos2))
                G.add_edge(road_id, eid, weight=weight)
    return G

def simplify_graph_by_junction(G):
    """
    对图 G 进行简化：
      将所有 junction 属性不为 "-1" 的节点合并为 aggregated 节点，
      其中 aggregated 节点的位置取所有原节点 position 的平均值。
      同时保留非 junction 节点之间的原有边。
    返回简化后的图 simplified_G。
    """
    groups = {}
    for node, data in G.nodes(data=True):
        junction = data.get("junction", "-1")
        if junction != "-1":
            groups.setdefault(junction, []).append(node)
    simplified_G = nx.DiGraph()
    # 添加非 junction 节点
    non_group_nodes = [node for node, data in G.nodes(data=True) if data.get("junction", "-1") == "-1"]
    for node in non_group_nodes:
        simplified_G.add_node(node, **G.nodes[node])
    # 将非 junction 内部的边也复制过来
    for u, v, d in G.edges(data=True):
        if u in non_group_nodes and v in non_group_nodes:
            simplified_G.add_edge(u, v, **d)
    # 对每个 junction 分组，生成 aggregated 节点
    for junc, nodes in groups.items():
        positions = [np.array(G.nodes[n]['pos']) for n in nodes if 'pos' in G.nodes[n]]
        if positions:
            center = tuple(np.mean(positions, axis=0))
        else:
            center = (0, 0)
        agg_node = f"Junction_{junc}"
        simplified_G.add_node(agg_node, pos=center, junction=junc)
        # 重写边：对于组内节点的出边和入边，从外部节点到组内的边替换成到 aggregated 节点
        for n in nodes:
            for u, v, d in G.out_edges(n, data=True):
                if v in nodes:
                    continue
                else:
                    simplified_G.add_edge(agg_node, v, **d)
            for u, v, d in G.in_edges(n, data=True):
                if u in nodes:
                    continue
                else:
                    simplified_G.add_edge(u, agg_node, **d)
    return simplified_G

def visualize_topology_combined(roads_data, detailed=True):
    """
    使用 networkx 和 Plotly 可视化路网拓扑。
    参数 detailed 控制显示详细程度：
      - detailed=True：显示所有 road 节点（节点标签显示 road_id 及 junction 信息）；
      - detailed=False：对 junction 路段节点合并为 aggregated 节点，
           其中 aggregated 节点位置为该 junction 内所有节点采样中间点的几何中心。
           其余非 junction 节点保持原样，所有边均根据原始连接重构。
    """
    # 构建原始图（适用于 Dijkstra 算法）
    G = build_topology_graph(roads_data)
    if not detailed:
        G = simplify_graph_by_junction(G)
    
    pos = nx.get_node_attributes(G, 'pos')
    
    edge_x, edge_y = [], []
    for u, v in G.edges():
        x0, y0 = pos[u]
        x1, y1 = pos[v]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
    edge_trace = go.Scatter(
        x=edge_x, y=edge_y,
        line=dict(width=1, color='#888'),
        hoverinfo='none',
        mode='lines'
    )
    
    node_x, node_y, node_text, node_color = [], [], [], []
    for node in G.nodes():
        x, y = pos[node]
        node_x.append(x)
        node_y.append(y)
        node_text.append(str(node))
        node_color.append('skyblue' if "Junction" not in str(node) else 'orange')
    node_trace = go.Scatter(
        x=node_x, y=node_y,
        mode='markers+text',
        text=node_text,
        textposition="top center",
        hoverinfo='text',
        marker=dict(
            showscale=False,
            color=node_color,
            size=20,
            line_width=2
        )
    )
    
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title=dict(text='路网拓扑图', font=dict(size=16)),
                        showlegend=False,
                        hovermode='closest',
                        margin=dict(b=20, l=5, r=5, t=40),
                        xaxis=dict(scaleanchor="y", scaleratio=1, showgrid=False, zeroline=False, showticklabels=False),
                        yaxis=dict(scaleanchor="x", scaleratio=1, showgrid=False, zeroline=False, showticklabels=False)
                    ))
    fig.show()

#%%
# ## 示例使用
#
# 这里我们先使用模拟数据构造一个 oxdr 地图数据。
# 若有实际的文件，可使用 load_oxdr_map 加载文件内容。
#
# 模拟数据格式参考上述假设

# 模拟的 oxdr 数据
file_path = "./road_map/parking1.xodr"
roads_data = parse_oxdr_all(file_path)

# 设置当前位置与感知范围
current_position = (15.0, 413.0)  # 例如：x=100米, y=200米
sensing_range = 30.0  # 例如：50米感知半径

# 提取感知范围内的车道
extract_lanes_in_range(roads_data, current_position, sensing_range)

# 可视化结果
visualize_lanes(roads_data, current_position, sensing_range)


# 调用可视化拓扑关系（可选）
visualize_topology_combined(roads_data, detailed=False)